In [2]:
import ot
import geomloss
import numpy as np
from ot.lp import emd_c

np.random.seed(114514)

In [7]:
a=np.array([.5,.5])
b=np.array([.5,.5])
M=np.array([[0.,1.],[1.,0.]])
print(ot.emd(a, b, M))
G, cost, _, _, result_code = emd_c(a, b, M, 1000000, 0)
print(cost)
print(G)


[[0.5 0. ]
 [0.  0.5]]
0.0
[[0.5 0. ]
 [0.  0.5]]
